In [5]:
from google.auth.transport.requests import Request
from sagemaker.s3 import S3Uploader, S3Downloader
from scipy.io.wavfile import write
from google.auth import default
from pydub import AudioSegment
from random import choice
from pathlib import Path

import numpy as np
import requests
import base64
import json
import os

In [2]:
!pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [3]:
%%bash
sudo apt-get update
sudo apt install -y ffmpeg

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1383 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1243 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [55.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4154 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2788 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4000 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages 

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  alsa-topology-conf alsa-ucm-conf dbus fontconfig fontconfig-config
ver intel-media-va-driver libaacs0 libaom3
cns0 libavc1394-0basound2 libasound2-data libass9 libasyn
  libavcodec58 libavdevice58 libavfilter7 libavformat58 libavutil56 libbdplus0
ibblas3 libbluray2 libbs2b0 libcaca0 libcairo-gobject2 libcairo2
ibcdio19 libchromaprint1 libcodec2-1.0
  libdatrie1 libdav1d5 libdbus-1-3 libdc1394-25 libdecor-0-0
  libdecor-0-plugin-1-cairo libdeflate0 libdrm-amdgpu1 libdrm-common
 libdrm-radeon1 libdrm2 libelf1 libflac8
  libflite1 libfontconfig1 libfreetype6 libfribidi0 libgbm1
  libgdk-pixbuf-2.0-0 libgdk-pixbuf2.0-bin libgdk-pixbuf2.0-common
er-dri libgl1-mesa-dri libglapi-mesa
ibgme0lib2.0-0 libglib2.0-data libglvnd0 libglx-mesa0 libglx0 l
  libgraphite2-3 libgsm1 libharfbuzz0b libiec61883-0 libigdgmm12
libjpeg-turbo8 libjpeg8 liblapack3 liblilv-0-0
1

debconf: delaying package configuration, since apt-utils is not installed


Fetched 145 MB in 5s (31.6 MB/s)
Selecting previously unselected package libapparmor1:amd64.
(Reading database ... 14553 files and directories currently installed.)
Preparing to unpack .../000-libapparmor1_3.0.4-2ubuntu2.4_amd64.deb ...
Unpacking libapparmor1:amd64 (3.0.4-2ubuntu2.4) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../001-libdbus-1-3_1.12.20-2ubuntu4.1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.12.20-2ubuntu4.1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../002-dbus_1.12.20-2ubuntu4.1_amd64.deb ...
Unpacking dbus (1.12.20-2ubuntu4.1) ...
Selecting previously unselected package libelf1:amd64.
Preparing to unpack .../003-libelf1_0.186-1ubuntu0.1_amd64.deb ...
Unpacking libelf1:amd64 (0.186-1ubuntu0.1) ...
Selecting previously unselected package libfribidi0:amd64.
Preparing to unpack .../004-libfribidi0_1.0.8-2ubuntu3.1_amd64.deb ...
Unpacking libfribidi0:amd64 (1.0.8-2ubuntu3.1) ...
Selecting previously uns

In [6]:
book_bucket = 's3://book-text-info'
book_dir = Path('book-text-info')
book_dir.mkdir(exist_ok=True)
downloader = S3Downloader()
downloader.download(book_bucket, book_dir)

['book-text-info/The-Bible/The-Bible-book-audio-text.txt',
 'book-text-info/The-Bible/The-Bible-character-dialogue.txt',
 'book-text-info/alice-in-wonderland/alice-in-wonderland-book-audio-text-tagged.txt',
 'book-text-info/alice-in-wonderland/alice-in-wonderland-book-audio-text.txt',
 'book-text-info/alice-in-wonderland/alice-in-wonderland-character-dialogue.txt',
 'book-text-info/alice-in-wonderland/alice-in-wonderland-person-quotes.json',
 'book-text-info/chapter2/chapter2-book-audio-text-tagged.txt',
 'book-text-info/chapter2/chapter2-book-audio-text.txt',
 'book-text-info/chapter2/chapter2-character-dialogue.txt',
 'book-text-info/chapter4/chapter4-book-audio-text-tagged.txt',
 'book-text-info/chapter4/chapter4-book-audio-text.txt',
 'book-text-info/chapter4/chapter4-character-dialogue.txt',
 'book-text-info/chapter4/chapter4-person-quotes.json',
 'book-text-info/chapter4/working-voices.json',
 'book-text-info/example_book/example_audio_text.txt',
 'book-text-info/example_book/exa

In [7]:
narr_bucket = 's3://sound-scribe-acessories/narrators.txt'
downloader.download(narr_bucket, '.')

['./narrators.txt']

In [8]:
files = list(book_dir.rglob('*-tagged.txt'))
files

[PosixPath('book-text-info/alice-in-wonderland/alice-in-wonderland-book-audio-text-tagged.txt'),
 PosixPath('book-text-info/chapter2/chapter2-book-audio-text-tagged.txt'),
 PosixPath('book-text-info/chapter4/chapter4-book-audio-text-tagged.txt'),
 PosixPath('book-text-info/frankenstein/frankenstein-book-audio-text-tagged.txt')]

In [9]:
# !export GOOGLE_CLOUD_PROJECT="mlops-final-tts"
os.environ.setdefault('GOOGLE_CLOUD_PROJECT', "mlops-final-tts")
for file in files:
    if '2' in file.name: continue
    with open(file) as f:
        ssml = ''.join(f.readlines())

    with open('narrators.txt') as f:
        voices = f.readlines()

    name = choice(voices).strip()
    payload = {
        "input": {
            "ssml": ssml
        },
        "audioConfig": {
            "audioEncoding": "LINEAR16"
        },
        "voice": {
            "languageCode": "en-US",
            "name": name
        },
        "outputGcsUri": f"gs://mlops-final-audio/{file.name.removesuffix('.txt')}.mp3"
    }
    # Force token refresh
    credentials, project = default()
    credentials.refresh(Request())

    if credentials.token:
        headers = {
            "Authorization": f"Bearer {credentials.token}",
            "X-Goog-User-Project": "mlops-final-tts"
        }
        # print("Headers:", headers)
        response = requests.post('https://texttospeech.googleapis.com/v1beta1/projects/mlops-final-tts/locations/global:synthesizeLongAudio', headers=headers, json=payload)
        try:
            with open(f'{file.name.removesuffix(".txt")}.response', 'w') as f:
                json.dump(response.json(), f)

        except Exception as e:
            print(e)
    else:
        print("Authentication failed. Verify service account setup.")